# Heasoft Tutorial

We select the GRB GRB 130831A from https://www.rochesterastronomy.org/snimages/grblist.html. The search resolves several options, and we select the one with the largest exposures in our desired X-ray and UV/Optical bands (second row).

![alt text](media/GRB%20130831A_exposure_selection.png "Title")

We then download the XRT event and UVOT images:

![](media/download_event.png) ![](media/download_images.png)

We also download the Swift Response Matrices for the XRT PC mode and UVOT filters, and place them in the directory for XRT and UVOT files respectively:

![](media/response_matrices.png)

Next we create `.reg` files for all our UVOT images and for the XRT event.

**XRT**:

<div style="display: flex; justify-content: space-around;">
  <img src="media/xrt_img.png" width="600"/>
</div>

<br>
<br>


**UVOT** (UBB, UVV, UUU, UM2, UW1, UW2 respectively):

<div style="display: flex; justify-content: space-around;">
  <img src="media/ubb_img.png" width="400"/>
  <img src="media/uvv_img.png" width="400"/>
  <img src="media/uuu_img.png" width="400"/>
</div>

<br>
<br>

<div style="display: flex; justify-content: space-around;">
  <img src="media/um2_img.png" width="400"/>
  <img src="media/uw1_img.png" width="400"/>
  <img src="media/uw2_img.png" width="400"/>
</div>

## Working on XRT event

From now on, we will work on the Sci Server on an image with Heasoft already installed. We extract the spectrum as follows:

```
xselect

read event sw01186032001xpcw3po_cl.evt

set image sky
filter region xrt_src.reg
extract spectrum
save spectrum xrt_src.pha

clear region

filter region xrt_bkg.reg
extract spectrum
save spectrum xrt_bkg.pha

exit
```


And then link the response files:

```
grppha
xrt_src.pha
xrt.pi
bad 0-29
group min 20
chkey RESPFILE swxpc0to12s6_20130101v014.rmf
chkey ANCRFILE swxs6_20010101v001.arf
chkey BACKFILE xrt_bkg.pha
exit
```

## Working on UVOT files

We summ up all images within each filter's image file:

```
uvotimsum sw00568849001uw1_sk.img uw1_sum.fits chatter = 1
uvotimsum sw00568849001uw2_sk.img uw2_sum.fits chatter = 1
uvotimsum sw00568849001uuu_sk.img uuu_sum.fits chatter = 1
uvotimsum sw00568849001ubb_sk.img ubb_sum.fits chatter = 1
uvotimsum sw00568849001uvv_sk.img uvv_sum.fits chatter = 1
uvotimsum sw00568849001um2_sk.img um2_sum.fits chatter = 1
```

Then we link the response files to extract the `.pha` files for these images:

```
uvot2pha infile=ubb_sum.fits srcpha=ubb.pha bkgpha=ubb_bkg.pha srcreg=ubb_src.reg bkgreg=ubb_bkg.reg respfile=swubb_20041120v105.rsp clobber=y chatter=1
uvot2pha infile=um2_sum.fits srcpha=um2.pha bkgpha=um2_bkg.pha srcreg=um2_src.reg bkgreg=um2_bkg.reg respfile=swum2_20041120v106.rsp clobber=y chatter=1
uvot2pha infile=uuu_sum.fits srcpha=uuu.pha bkgpha=uuu_bkg.pha srcreg=uuu_src.reg bkgreg=uuu_bkg.reg respfile=swuuu_20041120v105.rsp clobber=y chatter=1
uvot2pha infile=uvv_sum.fits srcpha=uvv.pha bkgpha=uvv_bkg.pha srcreg=uvv_src.reg bkgreg=uvv_bkg.reg respfile=swuvv_20041120v105.rsp clobber=y chatter=1
uvot2pha infile=uw1_sum.fits srcpha=uw1.pha bkgpha=uw1_bkg.pha srcreg=uw1_src.reg bkgreg=uw1_bkg.reg respfile=swuw1_20041120v106.rsp clobber=y chatter=1
uvot2pha infile=uw2_sum.fits srcpha=uw2.pha bkgpha=uw2_bkg.pha srcreg=uw2_src.reg bkgreg=uw2_bkg.reg respfile=swuw2_20041120v106.rsp clobber=y chatter=1
```

## Plotting and Fitting

We move all our files in the XRT and UCOT directories to a new one called `final_spec` to do our plotting and fitting. We are ready to plot with xspec:

```
xspec
data 1:1 uuu.pha 1:2 ubb.pha 1:3 uvv.pha 1:4 uw1.pha 1:5 uw2.pha 1:6 um2.pha 2:7 xrt.pi

cpd /png # we don't have a local installation of Heasoft
setplot en
plot ldata
```

<div style="display: flex; justify-content: space-around;">
  <img src="media/pgplot_pre_ignore.png" width="600"/>
</div>

`ignore bad`

<div style="display: flex; justify-content: space-around;">
  <img src="media/1.png" width="600"/>
</div>

```
model phabs*powerlaw

1:data group 1::phabs:nH>1
2:data group 1::powerlaw:PhoIndex>
3:data group 1::powerlaw:norm>

4:data group 2::phabs:nH>2
5:data group 2::powerlaw:PhoIndex>1.5
6:data group 2::powerlaw:norm>1e4
```

```
renorm
plot
```

<div style="display: flex; justify-content: space-around;">
  <img src="media/3.png" width="600"/>
</div>

```
fit 100

plot ldata
```

<div style="display: flex; justify-content: space-around;">
  <img src="media/5.png" width="600"/>
</div>

```
plot ldata residual
```

<div style="display: flex; justify-content: space-around;">
  <img src="media/4.png" width="600"/>
</div>

```
iplot

la x Energy (keV)
la y Counts s=1keV=1
la top SED of GRB 130831A
time off

color 1 on 13
color 2 on 14

plot
hardcopy grb_sed.ps/cps
```

<div style="display: flex; justify-content: space-around;">
  <img src="media/0.png" width="600"/>
</div>